In [ ]:
#####################################################################
#
# setup
#
#####################################################################

In [ ]:
# imports for this notebook to run
import sys
from datetime import datetime
from typing import NamedTuple

from google.cloud import aiplatform as vertex
from google_cloud_pipeline_components.experimental import vertex_notification_email as gcc_exp

import kfp
from kfp.v2 import dsl, compiler
from kfp.v2.dsl import (Artifact, Dataset, Input, InputPath, Model, Output, OutputPath, component)

In [35]:
# specify parameters
PROJECT_ID = "your-project-id"
REGION = "us-central1"
BUCKET_NAME = f"bkt-{PROJECT_ID}-vpipelines"
BUCKET_PATH = f"gs://{BUCKET_NAME}"
PIPELINE_ROOT = f"{BUCKET_PATH}/pipeline_root"
PIPELINE_DATA = f"{BUCKET_PATH}/data"
TIMESTAMP = datetime.now().strftime("%Y%m%d%H%M%S")

In [ ]:
#####################################################################
# BEGIN vertex pipelines
#####################################################################

In [ ]:
#####################################################################
#
# create individual pipeline components, then specify the pipeline
#
#####################################################################

In [46]:
# component producing only one output
@component(base_image="python:3.9", output_component_file="component_one_output.yaml")
# specify inputs, outputs, and component functionality
def one_output(text: str) -> str:
    print(text)
    return text

In [47]:
# component producing multiple outputs
@component(base_image="python:3.9", output_component_file="component_two_outputs.yaml")
# specify inputs, outputs, and component functionality
def two_outputs(
    text: str, # inputs and their expected types
    ) -> NamedTuple(
    "Outputs",
    [
        ("output_one", str),  # outputs and their expected types
        ("output_two", str),
    ],
    ):
    # component functionality
    o1 = f"output one from text: {text}"
    o2 = f"output two from text: {text}"
    return (o1, o2)

In [48]:
# consume outputs from other components as inputs to this component
@component(base_image="python:3.9", output_component_file="component_consumer.yaml")
def consumer(text1: str, text2: str, text3: str):
    print(f"text1: {text1}; text2: {text2}; text3: {text3}")

In [49]:
# define a pipeline
@dsl.pipeline(name="my-pipeline-name", description="my pipeline description")

# specify all the inputs the pipeline needs to run
def my_pipeline(text: str
                , project_id: str = PROJECT_ID):
    
    # notification recipients
    RECIPIENTS_LIST = ["your-email@your-domain.com"]
    notify_email_task = gcc_exp.VertexNotificationEmailOp(recipients=RECIPIENTS_LIST)
    
    # when pipeline exits, send status notification
    with dsl.ExitHandler(notify_email_task):
        
        # specify the nodes in the pipeline
        one_output_task = one_output(text)
        two_outputs_task = two_outputs(text)
        consumer_task = consumer(
            # notice the difference in accessing items
            one_output_task.output,
            two_outputs_task.outputs["output_one"],
            two_outputs_task.outputs["output_two"],
        )

In [ ]:
#####################################################################
#
# compile and run the pipeline
#
#####################################################################

In [50]:
# compile the pipeline
my_package_path = 'my_vertex_pipeline_specification_file.json'
compiler.Compiler().compile(pipeline_func=my_pipeline, package_path=my_package_path)

In [ ]:
# runtime parameters to pass to pipeline
pipeline_params = {
    "project_id": PROJECT_ID,
    "text": "hello"
}

# run the pipeline
vertex.init(project=PROJECT_ID)

job = vertex.PipelineJob(
    display_name = "my-pipeline-job-name",
    template_path = my_package_path,
    pipeline_root = PIPELINE_ROOT,
    parameter_values = pipeline_params,
    enable_caching = False
)

job.run()